In [5]:
import numpy as np
from pyfigtree import figtree

In [6]:
def position(X,alpha,delta):
    X_x = figtree(X,X,alpha[:,0], delta,epsilon=1e-16, eval="auto")
    X_y = figtree(X,X,alpha[:,1], delta,epsilon=1e-16, eval="auto")
    X_z = figtree(X,X,alpha[:,2], delta,epsilon=1e-16, eval="auto") 
    return np.dstack([X_x,X_y, X_z])[0]

In [7]:
def momentum(X,alpha,delta):
    ax = alpha[:,0]
    ay = alpha[:,1]
    az = alpha[:,2]
    
    first_weights = np.dstack([ax * X[:,0], ax * X[:,1], ax * X[:,2]])[0]
    second_weights = np.dstack([ay * X[:,0], ay * X[:,1], ay * X[:,2]])[0]
    third_weights = np.dstack([az * X[:,0], az * X[:,1], az * X[:,2]])[0]
    
    first_sum_x = figtree(X,X,first_weights[:,0],delta, epsilon=1e-16, eval="auto")
    first_sum_y = figtree(X,X,first_weights[:,1],delta, epsilon=1e-16, eval="auto")
    first_sum_z = figtree(X,X,first_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    second_sum_x = figtree(X,X,second_weights[:,0],delta, epsilon=1e-16, eval="auto")
    second_sum_y = figtree(X,X,second_weights[:,1],delta, epsilon=1e-16, eval="auto")
    second_sum_z = figtree(X,X,second_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    
    third_sum_x = figtree(X,X,third_weights[:,0],delta, epsilon=1e-16, eval="auto")
    third_sum_y = figtree(X,X,third_weights[:,1],delta, epsilon=1e-16, eval="auto")
    third_sum_z = figtree(X,X,third_weights[:,2],delta, epsilon=1e-16, eval="auto")
    
    fourth_sum = figtree(X,X,ax,delta, epsilon=1e-16, eval="auto")
    fifth_sum = figtree(X,X,ay,delta, epsilon=1e-16, eval="auto")
    sixth_sum = figtree(X,X,az,delta, epsilon=1e-16, eval="auto")
    
    second_term = fourth_sum + fifth_sum + sixth_sum
    
    second_term_x = X[:,0] * second_term
    second_term_y = X[:,1] * second_term
    second_term_z = X[:,2] * second_term
 
    mult = -2./X.shape[0]
    result_x = mult*(ax* first_sum_x + ay * second_sum_x + az * third_sum_x - (second_term_x) )
    result_y = mult*(ax* first_sum_y + ay * second_sum_y + az * third_sum_y - (second_term_y) )
    result_z = mult*(ax* first_sum_z + ay * second_sum_z + az * third_sum_z - (second_term_z) )
    result = np.dstack([result_x,result_y,result_z])[0]
    
    return result

In [8]:
def f(vector, delta):
    positions, momentums = np.split(vector,2)
    positions = np.reshape(positions,(3,len(positions)/3)).T
    momentums = np.reshape(momentums,(3,len(momentums)/3)).T
    new_positions = position(positions, momentums,delta)
    new_momentums = momentum(positions, momentums,delta)
    n,m = new_positions.shape
    new_positions = np.reshape(new_positions.T, (n*m))
    new_momentums = np.reshape(new_momentums.T, (n*m))
    return np.hstack([new_positions,new_momentums])